# Scan Path Analysis pt3
#### @tutor: Ms SHARAFI Zohreh
#### @student: Mr SHAW Oscar

#### In this notebook, we will study scan-path through the ScanMatch Algorithm, the Needleman-Wunsch algorithm will be use to compare the similarity of two sequences of AOI. 
As in the previous studies, the first step will be the preprocessing of the data. We will then clean data from misformed string and None values in order to traduce entity sequences into letter sequences.
Moreover, in order to artificially take fixation duration into account, a letter will be duplicated in proportion to the duration of the fixation, according to a coefficient, being the average duration of a fixation, ie 20 to 30ms, say 25ms. (Rayner, 1978). Thus, if a fixation on AOI corresponding to the letter A last for 100ms, we will traduce this by "AAAA".
This done, we will thus be able to apply the algorithm of Needleman-Wunsch in order to calculate the score of similarity between the different code entities. Thus, failing to visualize whether the sequences are "geographically" similar, we can at least decide on the logic of the sequences.

One thing that can be interesting in this is to compare the sequences of the same task between the different participants. Indeed, if the comparison of the sequences of the different tasks for the same candidate could have been relevant in a previous study, when only the successions of entities were studied, it will not be reliable here because a task can be more difficult than another and this can impact the duration of fixation and therefore the relevance of the study.

In [1]:
# import libraries
# !pip install pandas

import os
import pandas as pd
import numpy as np

## 1- Data Import

In [2]:
# Path of the folder which contains the files
FOLDER_PATH = "C:\\Users...."
DATA_FOLDER = os.path.join(FOLDER_PATH, 'data_files')

# Dictionary establishing the correspondence between a file and its sequence of entities
# i.e data{key = file_id, value = [seq. of entities]} where entities are in [‘Comment’, ‘Bug_Report’, ‘Member_Variable’, 
# ‘Method_Body’, ‘Method_Signature’]
# ex: file_id = P_103 & value = ['Comment','Bug_Report','None','Bug_Report']
data = {}


# For each csv files, we use the dataframe to file the dictionnary
for filename in os.listdir(DATA_FOLDER):
    df = pd.read_csv(os.path.join(DATA_FOLDER, filename),delimiter=',')
    data[filename.split('_')[0]] = df

## 2- Cleaning & Preprocessing


In [7]:
# Divide the DF regarding the phase number
def divide(df,features,i):
#     input:  df,i  -> object (dataframe) large dataframe with all phase number, int number of phase_number
#     output: df -> object (dataframe) array of df depending on the phase number 
    inp = [df.loc[df['phase_number'] == i+1, features] for i in range(i)]
    return inp

In [8]:
# Allow to clean NA values
def cleanNA(df):
#     input:  df  -> object (dataframe) with NA  
#     output: df -> object (dataframe) without NA and reindexed 
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

In [9]:
def convert(df):
    # Converts kind of entity to letter then duplicates the letter
    df['entity'] = df['entity'].map(lambda x: 'A' if x.lower() == 'method_body' else ('B' if x.lower() == 'comment' else ('C' if x.lower() == 'member_variable' else ('D' if x.lower() == 'bug_report' else ('E' if x.lower() == 'method_signature' else pd.NA)))))
    df = cleanNA(df)
    df['entity'] =  (df['fix_dur'] // 25000000) * df['entity']
    return df

In [13]:
# Transform the sequence of letter into a string for latter treatments
def SequenceToString(df):
#     input: df -> object (dataframe)
#     output: string -> string which embodies the sequence of entities with letters
    return ''.join( ''.join(map(str, x)) for x in df.entity)

In [14]:
# Iterate over each dataframes of data and clean data in each, then convert the data.
# There we use another dictionnary for more reusability of past data
def pipeline(data):
#      input: data, features  -> dictionnary{key = file_id, value = object (dataframe)}, array of string containing the feature we want to keep
#      output: new_data -> dictionnary{key = file_id, value = [sequency]}   
    new_data = {}
    for i in data:
        divided_df = divide(data[i],['entity','fix_dur'],max(data[i]['phase_number']))
        res = []
        for df in divided_df:
            df = convert(df)
            df = SequenceToString(df)      
            res.append(df)
        new_data[i] = res
    return new_data

new_data = pipeline(data)
new_data

{'P102': ['BBBBBBDDDDBBBBBBBBCCCCCCCCCBBBDDDDDDDDDDDDDDDDDDDBBBBBBBCCCBBBBBBBBBBBBBBBBBBBBBBBBBAABBBBBEEEEEBBBBBBBBBAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABBBBAAAAAAAAAAAAAAABBBBBBBBBBBBBBBAAAAABBBBBBBBBBBBBBBBBBBBBAAAAABBBBBBBBBBBBBBBBAAAAAAAAAAAAAABBBBBBBBBBBBBBBBBBBBBBBBBAAAAAAAAAAAAAAAAAAAAAAAABBBBBBBBBBBBBBBBBBBBBAAAAABBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBAAAAAAABBBBBBBBBBBBBBBBBAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABBBBBBBBBBBBBBBBBBBBBAAAAAAAAAAAAAAAAAAACCCCCCCCCCCCCBBAAAAAABBBBBBBBBBBBAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABBBBBBBAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

### Then we will implement the Needleman-Wunsch algorithm.
 This algorithm allows, given a string x and a string y, to find the optimal combination of operations to transform x into y. The optimal combination means, in a nutshell, the one which allows to find seq. of steps to get from x to y, with the highest score, being clear that the more sequences are similar the higher the score. The allowed operations are: insert, delete or replace. 

Let's understand how it works:

The approach adopted in this algorithm is the one of dynamical programming. We assume that solving this problem is made by focusing on subproblems, which are calculing the cost to align to elements for each elements & then, finding the better way to reach the top.

NB: when two ways are possible, we consider delete or insert better than gap since match is preferred

An important point is that this algorithm allows two things, find the optimal score & find the sequence of operation to get from x to y. However, we'll only focus on the first aspect since we only study similarity & that the next aspect will be useless & lead to lower performances

To compare all those elements we will use a matrix in which rows are m[i][j] is the minimum cost to align the i first element of string x to the j first of y. Thus by recursivity we can use the following formula:

m[i][j] = min(cost_by_align,cost_by_delete,cost_by_insert)
 
##### m[i][j] = min((m[i-1][j-1]+Cij),(m[i-1][j]+1),(m[i][j-1]+1)) where Cij = 1 if xi != xj else 0

In [15]:
def needleman_wunsch_score(s1: str, s2: str, match = 3, mismatch = -2, gap = -1):   
#      input: s1, s2  -> str, str: the two strings that are compared 
#      output: normalized_score -> float: normalized score between the strings
    
    #Dict which gaves the score regarding s1[i] & s2[j] 
    score_dict = {}
    for i in set(s1): # set to have better performance since string are longs with lots of same letters
        for j in set(s2):
            if i == j:
                score_dict[i+j] = match
            else:
                score_dict[i+j] = mismatch
    
    score_matrix = np.zeros((len(s1)+1,len(s2)+1),int) #we define a matrix which we contains the scores
    score_matrix[:,0] = np.linspace(0, gap * len(s1),len(s1)+1)
    score_matrix[0,:] = np.linspace(0, gap * len(s2),len(s2)+1)

    for i in range(1,len(s1)+1):
        for j in range(1,len(s2)+1):
            #score = max(             new score by alignement                       new score by deletion     new score by insertion )                                         )
            score_matrix[i,j] = max(score_matrix[i-1,j-1]+score_dict[s1[i-1]+s2[j-1]],score_matrix[i-1,j]+gap,score_matrix[i,j-1]+gap)
    distance = score_matrix[-1,-1]
    normalized_score = 1/ (1+np.exp(-distance / (max(len(s1),len(s2)))))
    return normalized_score


In [17]:
alreadyMade = []
correlated_res = {}
for x in new_data.items():
    for y in new_data.items():
        partialsSeq = []
        for i in range(len(x[1])):
            if y not in alreadyMade and i< len(y[1]) and x[0]!=y[0] and x[1][i] and y[1][i]:
                partialsSeq.append((i+1,round(needleman_wunsch_score(x[1][i],y[1][i]),5)))
        if partialsSeq:
            correlated_res['_'.join((x[0],y[0]))] = partialsSeq
    alreadyMade.append(x)
correlated_res

{'P102_P103': [(1, 0.53627), (2, 0.84668), (3, 0.32638)],
 'P102_P105': [(1, 0.83879), (2, 0.87134)],
 'P102_P106': [(2, 0.63637), (3, 0.63944)],
 'P102_P107': [(1, 0.34916), (2, 0.80086), (3, 0.48446)],
 'P103_P105': [(1, 0.50918), (2, 0.86234)],
 'P103_P106': [(2, 0.57605), (3, 0.40501)],
 'P103_P107': [(1, 0.51852), (2, 0.83975), (3, 0.28206)],
 'P105_P106': [(2, 0.60628)],
 'P105_P107': [(1, 0.3317), (2, 0.81874)],
 'P106_P107': [(2, 0.52408), (3, 0.36058)]}